<a href="https://colab.research.google.com/github/ihagoSantos/image-analysis-and-computer-vision/blob/main/keras_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação de filmes

Nesse exemplo, é mostrado a construção de um classificador binário para o dataset IMDB (NLP Problem).\
\
O imdb dataset já vem com o Keras e já é pré-processado. Os reviews (sequência de palavras já estao organizados em sequências de inteiros, em que cada inteiro significa uma palavra específica no dicionario)

In [17]:
from keras.datasets import imdb

As 10000 palavras mais frequentes serão mantidas no dataset

In [18]:
# class load_data with allow_pickle implicity set to true
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

train_data e test_data são as listas de reviews, cada review é uma lista de índices de palavras (textos do review codificado no índice de palavras)

In [19]:
print(train_data[6])

[1, 6740, 365, 1234, 5, 1156, 354, 11, 14, 5327, 6638, 7, 1016, 2, 5940, 356, 44, 4, 1349, 500, 746, 5, 200, 4, 4132, 11, 2, 9363, 1117, 1831, 7485, 5, 4831, 26, 6, 2, 4183, 17, 369, 37, 215, 1345, 143, 2, 5, 1838, 8, 1974, 15, 36, 119, 257, 85, 52, 486, 9, 6, 2, 8564, 63, 271, 6, 196, 96, 949, 4121, 4, 2, 7, 4, 2212, 2436, 819, 63, 47, 77, 7175, 180, 6, 227, 11, 94, 2494, 2, 13, 423, 4, 168, 7, 4, 22, 5, 89, 665, 71, 270, 56, 5, 13, 197, 12, 161, 5390, 99, 76, 23, 2, 7, 419, 665, 40, 91, 85, 108, 7, 4, 2084, 5, 4773, 81, 55, 52, 1901]


train_labels e test_labels são as listas de 0's e 1's (0 = review negativo e 1 = review positivo)

In [26]:
print(train_labels[6])

1


Veja que nenhum índice de palavra irá exceder 10000

In [20]:
max([max(sequence) for sequence in train_data])

9999

Não podemos inserir uma lista de inteiros na rede. Temos que converter a lista para um tensor no formato (samples, word_indices).\
Por exemplo, transformar a sequência [3,5] em um vetor de 10.000 dimensões que seria todos os 0s, exceto os indices 3 e 5, que seria 1s.

In [21]:
import numpy as np

In [22]:
def vectorize_sequences(sequences, dimension=10000):
  # creates an all-zero matrix of shape (len(sequences), dimension)
  results = np.zeros((len(sequences), dimension))

  for i, sequence in enumerate(sequences):
    # Sets specific indices of results[i] to 1s
    results[i, sequence] = 1.

  return results

In [23]:
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [24]:
print(x_train[0])
print(x_train.shape)

[0. 1. 1. ... 0. 0. 0.]
(25000, 10000)


O mesmo para os vetores de labels

In [25]:
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

## Construindo a rede

O input é um vetor e os labels são escalares (1's e 0's). Vamos usar um fully connected (Dense) com relu como activation function. \
```
Dense(16, activation='relu')
```
16: é o argumento para cada dense layer. Ou seja, é o número de hidden units da camada (é a dimensão da representação da camada).

## Configurando a rede
Iremos configurar a rede com *duas camandas intermediárias* com *16 unidades ocultas* cada. *Uma terceira camada que produzirá a previsão* escalar em relação ao sentimento do review em questão.

In [27]:
from tensorflow.keras import models
from tensorflow.keras import layers


In [28]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Escolhendo a loss function e o optimizer (como strings por já fazerem parte do Keras)

In [30]:
from tensorflow.keras import losses
from tensorflow.keras import metrics
# from tensorflow.keras.optimizers import SGD

# model.compile(optimizer=RMSprop(lr=0.001))
# loss = losses.binary_crossentropy
# metrics = [metrics.binary_accuracy]

No entanto, você pode deixar tudo como padrão

In [31]:
model.compile(
    optimizer="rmsprop",
    loss="binary_crossentropy",
    metrics=["acc"]
)

Para monitorar o treinamento, criamos o conjunto de validação separando 10000 amostras do conjunto original

In [32]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

Iremos treinar o modelo por 20 épocas (20 iterações sobre todas as amostras do treinamento) em um batch de 512 amostras.

In [33]:
model.fit(
    partial_x_train,
    partial_y_train,
    epochs=100,
    batch_size=512,
    validation_data=(x_val, y_val)
)

Epoch 1/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 141ms/step - acc: 0.7117 - loss: 0.6077 - val_acc: 0.8128 - val_loss: 0.4622
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - acc: 0.8780 - loss: 0.3929 - val_acc: 0.8768 - val_loss: 0.3482
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - acc: 0.9145 - loss: 0.2808 - val_acc: 0.8853 - val_loss: 0.3006
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - acc: 0.9312 - loss: 0.2269 - val_acc: 0.8892 - val_loss: 0.2814
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - acc: 0.9445 - loss: 0.1868 - val_acc: 0.8869 - val_loss: 0.2761
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - acc: 0.9531 - loss: 0.1588 - val_acc: 0.8807 - val_loss: 0.2970
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - acc: 0.9605 - loss: 0.1394 - val_acc: 0.8841 - val_loss: 0.2862
Epoch 8/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - acc: 0.9680 - loss: 0.1170 - val_acc: 0.8857 - val_loss: 0.2875
Epoch 9/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - a

## Usando uma rede treinada para gerar previsões sobre novos dados
Gerando a probabilidade de cada análise ser positiva com o método predict

In [34]:
print(model.predict(x_test))

782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
[[7.6585575e-06]
 [1.0000000e+00]
 [9.5876306e-01]
 ...
 [4.1779423e-08]
 [3.2187970e-06]
 [9.9401641e-01]]


# Treinando o modelo do zero
Observe que no treino abaixo não separamos o conjunto de treino em treino e validação, uma vez que não iremos validar o modelo durante o treinamento e sim após o mesmo com o model.evaluate()

In [38]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [39]:
model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['acc']
)

In [40]:
model.fit(x_train, y_train, epochs=100, batch_size=512)
results = model.evaluate(x_test, y_test)

Epoch 1/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - acc: 0.7388 - loss: 0.5683
Epoch 2/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - acc: 0.9028 - loss: 0.2978
Epoch 3/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - acc: 0.9204 - loss: 0.2193
Epoch 4/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - acc: 0.9345 - loss: 0.1834
Epoch 5/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - acc: 0.9449 - loss: 0.1587
Epoch 6/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - acc: 0.9540 - loss: 0.1393
Epoch 7/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - acc: 0.9557 - loss: 0.1306
Epoch 8/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - acc: 0.9618 - loss: 0.1140
Epoch 9/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - acc: 0.9658 - loss: 0.1052
Epoch 10/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - acc: 0.9701 - loss: 0.0942
Epoch 11/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - acc: 0.9703 - loss: 0.0912
Epoch 12/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - acc: 0.9753 - loss: 0.0801
Epoch 13/100
49/49 ━━━━━━

In [41]:
results = model.evaluate(x_test, y_test)

782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - acc: 0.8477 - loss: 2.0736


In [42]:
results

[2.0652267932891846, 0.8471999764442444]

Referência: François Chollet. Deep Learning with Python. November 2017